In [ ]:
# Asthetics
import warnings
warnings.filterwarnings('ignore')
import time
# Basic
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np
import json
import os
import random
import string
import re
from functools import partial
import nltk


# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')#.sample(frac = 0.3)
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
train_df.head()

In [ ]:
def read_append_return(filename, train_files_path = train_files_path, output = 'text'):
    json_path = os.path.join(train_files_path, (filename + '.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
train_df['text'] = train_df['Id'].apply(read_append_return)

In [ ]:
train_df.shape

In [ ]:
train_df['cleaned_label'].nunique()

In [ ]:
train_df['text_len'] = train_df['text'].str.len()


In [ ]:
max_len_text = int(train_df['text_len'].quantile(0.90))

In [ ]:
max_len_text

In [ ]:
def one_type(word):
    letters = len([x for x in word if x.isalpha()])
    numbers = len([x for x in word if x.isnumeric()])
    return letters*numbers

def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[\"#/@;:<>{}`+=~|?,]", "", text)
    text = re.sub(r"\r", " ", text)
    text = re.sub(r"[.-]", " ", text)
    text = re.sub(r"[']", " ", text)
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    text = " ".join([x for x in text.split(' ') if (x != '')])
    text = " ".join([x for x in text.split(' ') if one_type(x) == 0])
    
    return text

def tagger(decoder_input_sentence):
    bos = "<BOS> "
    eos = " <EOS>"
    final_target = bos + decoder_input_sentence + eos
    return final_target

In [ ]:
datasets_titles = [x.lower() for x in set(train_df['dataset_title'].unique()).union(set(train_df['dataset_label'].unique()))]

In [ ]:
labels = []
for index in train_df['Id']:
    publication_text = train_df[train_df['Id'] == index].text.str.cat(sep='\n').lower()
    label = []
    for dataset_title in datasets_titles:
        if dataset_title in publication_text:
            label.append(clean_text(dataset_title))
    labels.append(' | '.join(label))

train_df['cleaned_label_test'] = labels

In [ ]:
train_df['cleaned_label_test'].head()

In [ ]:
train_df['cleaned_label_test'].apply(lambda x: len(x.split())).max()

In [ ]:
train_df['cleaned_label_test'] = train_df['cleaned_label_test'].apply(lambda x: tagger(x))
#train_df['cleaned_label'] = train_df['cleaned_label'].apply(lambda x: tagger(x))
train_df['cleaned_text'] = train_df['text'].apply(clean_text)
train_df['cleaned_text'] = train_df['cleaned_text'].str[:max_len_text]

In [ ]:
train_df['text'].apply(lambda x: len(x.split())).describe()

In [ ]:
train_df['cleaned_label_test'].apply(lambda x: len(x.split(' '))).hist()

In [ ]:
import numpy as np  
import pandas as pd 
import re           
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf

In [ ]:
max_len_text=512 
max_len_summary=25

In [ ]:
num_words = 5000

In [ ]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words = num_words)
x_tokenizer.fit_on_texts(list(train_df['cleaned_text']))

#convert text sequences into integer sequences
x    =   x_tokenizer.texts_to_sequences(train_df['cleaned_text']) 

#padding zero upto maximum length
x    =   pad_sequences(x,  maxlen=max_len_text, padding='post') 

x_voc_size   =  len(x_tokenizer.word_index) +1

In [ ]:
#preparing a tokenizer for summary on training data 
y_tokenizer = Tokenizer(num_words = num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{}~\t\n')
y_tokenizer.fit_on_texts(list(train_df['cleaned_label_test']))

#convert summary sequences into integer sequences
y    =   y_tokenizer.texts_to_sequences(train_df['cleaned_label_test']) 

#padding zero upto maximum length
y    =   pad_sequences(y, maxlen=max_len_summary, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

In [ ]:
import keras

In [ ]:
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
from keras import backend as K 
K.clear_session() 
latent_dim = 256
drop_rate = 0.5

# Encoder 
encoder_inputs = Input(shape=(max_len_text,)) 
enc_emb = Embedding(x_voc_size, latent_dim, trainable = True)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences = True, return_state = True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)
encoder_output1 = Dropout(drop_rate)(encoder_output1)

leakyrelu = tf.keras.layers.LeakyReLU()
encoder_output1 = leakyrelu(encoder_output1)

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences = True,return_state = True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 
encoder_output2 = Dropout(drop_rate)(encoder_output2)

leakyrelu = tf.keras.layers.LeakyReLU()
encoder_output2 = leakyrelu(encoder_output2)

#LSTM 3 
encoder_lstm3 = LSTM(latent_dim, return_state = True, return_sequences = True) 
encoder_output3, state_h3, state_c3 = encoder_lstm3(encoder_output2) 
encoder_output3 = Dropout(drop_rate)(encoder_output3)

leakyrelu = tf.keras.layers.LeakyReLU()
encoder_output3 = leakyrelu(encoder_output3)

#LSTM 4
encoder_lstm4 = LSTM(latent_dim, return_state = True, return_sequences = True) 
encoder_output4, state_h4, state_c4 = encoder_lstm4(encoder_output3) 
encoder_output4 = Dropout(drop_rate)(encoder_output4)

leakyrelu = tf.keras.layers.LeakyReLU()
encoder_output4 = leakyrelu(encoder_output4)

#LSTM 5
encoder_lstm5 = LSTM(latent_dim, return_state = True, return_sequences = True) 
encoder_output5, state_h5, state_c5 = encoder_lstm5(encoder_output4) 
encoder_output5 = Dropout(drop_rate)(encoder_output5)

leakyrelu = tf.keras.layers.LeakyReLU()
encoder_output5 = leakyrelu(encoder_output5)

#LSTM 6
encoder_lstm6 = LSTM(latent_dim, return_state = True, return_sequences = True) 
encoder_output6, state_h6, state_c7 = encoder_lstm6(encoder_output5) 
encoder_output6 = Dropout(drop_rate)(encoder_output6)

leakyrelu = tf.keras.layers.LeakyReLU()
encoder_output6 = leakyrelu(encoder_output6)

#LSTM 7
encoder_lstm7 = LSTM(latent_dim, return_state = True, return_sequences = True) 
encoder_outputs, state_h, state_c = encoder_lstm7(encoder_output6) 
encoder_outputs = Dropout(drop_rate)(encoder_outputs)

# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(y_voc_size, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM 1 using encoder_states as initial state
decoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_output1,decoder_fwd_state1, decoder_back_state1 = decoder_lstm1(dec_emb,initial_state=[state_h, state_c]) 
decoder_output1 = Dropout(drop_rate)(decoder_output1)

leakyrelu = tf.keras.layers.LeakyReLU()
decoder_outputs = leakyrelu(decoder_output1)

#LSTM 2
#decoder_lstm2 = LSTM(latent_dim, return_state = True, return_sequences = True) 
#decoder_output2, state_h2, state_c2 = decoder_lstm2(decoder_output1) 
#decoder_output2 = Dropout(drop_rate)(decoder_output2)

#leakyrelu = tf.keras.layers.LeakyReLU()
#decoder_outputs = leakyrelu(decoder_output2)

#LSTM 3
#decoder_lstm3 = LSTM(latent_dim, return_state = True, return_sequences = True) 
#decoder_output3, state_h3, state_c3 = decoder_lstm3(decoder_output2) 
#decoder_output3 = Dropout(drop_rate)(decoder_output3)

#leakyrelu = tf.keras.layers.LeakyReLU()
#decoder_output3 = leakyrelu(decoder_output3)

#LSTM 4
#decoder_lstm4 = LSTM(latent_dim, return_state = True, return_sequences = True) 
#decoder_outputs, state_h4, state_c4 = decoder_lstm4(decoder_output3) 
#decoder_outputs = Dropout(drop_rate)(decoder_outputs)


#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])
#attn_out = Dropout(drop_rate)(attn_out)


# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
history = model.fit([x,y[:,:-1]], y.reshape(y.shape[0],y.shape[1], 1)[:,1:] ,epochs=10,
                  batch_size=32, validation_split = 0.2)

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train') 
pyplot.plot(history.history['val_loss'], label='test') 
pyplot.legend()
pyplot.show()

In [ ]:
reverse_target_word_index=y_tokenizer.index_word 
reverse_source_word_index=x_tokenizer.index_word 
target_word_index=y_tokenizer.word_index

In [ ]:
# encoder inference
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h5, state_c5])

# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm1(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = target_word_index['bos']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        #print(sampled_token)
        #print(stop_condition)
        #print(sampled_token == 'eos')
        if (sampled_token!='eos'):
            decoded_sentence += ' '+sampled_token

            # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eos') or (len(decoded_sentence.split()) >= (max_len_summary-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

In [ ]:
sample_sub

In [ ]:
sample_sub['text'] = sample_sub['Id'].apply(lambda x: read_append_return(x, test_files_path))

In [ ]:
sample_sub['cleaned_text'] = sample_sub['text'].apply(clean_text)

In [ ]:
sub_seqs = x_tokenizer.texts_to_sequences(sample_sub['cleaned_text'])
sub_seqs = pad_sequences(sub_seqs, maxlen=max_len_text, padding='post') 

In [ ]:
predicted = []
for i in range(len(sub_seqs)):
    decoded_sequence = decode_sequence(sub_seqs[i].reshape(1, max_len_text)).strip().replace(' | ','|')
    print(decoded_sequence)
    predicted.append(decoded_sequence)

In [ ]:
sample_sub['PredictionString'] = predicted

In [ ]:
sample_sub = sample_sub.drop(['text','cleaned_text'], axis = 1)
sample_sub.head()

In [ ]:
sample_sub.to_csv('submission.csv', index = False)